**Задание 1. (5 баллов)**

Реализуйте триграмную модель и сгенерируйте несколько текстов. Сравните их с текстами, сгенерированными биграмной моделью. 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [2]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [ ]:
!pip install pymorphy2
!pip install pymystem3==0.1.10
!pip install rusenttokenize
!pip install razdel

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [37]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
dvach = open('2ch_corpus.txt').read()[:200000]
news = open('lenta.txt').read()[:200000]

In [6]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


In [7]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [8]:
print("Длина корпуса токсичных постов в токенах -", len(norm_dvach))
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))

Длина корпуса токсичных постов в токенах - 31781
Длина корпуса новостных текстов в токенах -  25515


In [9]:
print("Уникальных токенов в корпусе токсичных постов -", len(set(norm_dvach)))
print("Уникальный токенов в корпусе новостных текстов - ", len(set(norm_news)))

Уникальных токенов в корпусе токсичных постов - 9976
Уникальный токенов в корпусе новостных текстов -  9555


In [10]:
from nltk.tokenize import sent_tokenize

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [11]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [12]:
from collections import Counter

unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))

unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))


In [13]:
len(unigrams_dvach), len(bigrams_dvach), len(trigrams_dvach)

(9978, 26664, 30846)

Cоздадим матрицы с вероятностями для биграмм и триграмм

In [14]:
#bigrams + dvach
matrix_dvach = np.zeros((len(unigrams_dvach), len(unigrams_dvach)))
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

for ngram in bigrams_dvach:
    word1, word2 = ngram.split()
    matrix_dvach[word2id_dvach[word1]][word2id_dvach[word2]] = (bigrams_dvach[ngram]/unigrams_dvach[word1])

In [15]:
#bigrams + news
matrix_news = np.zeros((len(unigrams_news), len(unigrams_news)))
id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

for ngram in bigrams_news:
    word1, word2 = ngram.split()
    matrix_news[word2id_news[word1]][word2id_news[word2]] =  (bigrams_news[ngram]/unigrams_news[word1])

In [16]:
#trigrams + dvach
matrix_dvach_2 = np.zeros((len(bigrams_dvach), len(unigrams_dvach)))
#id2word_dvach = list(unigrams_dvach)
#word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}
id2bi_dvach = list(bigrams_dvach)
bi2id_dvach = {bi:i for i, bi in enumerate(id2bi_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_dvach_2[bi2id_dvach[bigram]][word2id_dvach[word3]] = (trigrams_dvach[ngram]/bigrams_dvach[bigram])

In [17]:
#trigrams + news
matrix_news_2 = np.zeros((len(bigrams_news), len(unigrams_news)))
#id2word_news = list(unigrams_news)
#word2id_news = {word:i for i, word in enumerate(id2word_news)}
id2bi_news = list(bigrams_news)
bi2id_news = {bi:i for i, bi in enumerate(id2bi_news)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_news_2[bi2id_news[bigram]][word2id_news[word3]] = (trigrams_news[ngram]/bigrams_news[bigram])

In [18]:
matrix_dvach.shape, matrix_news.shape, matrix_dvach_2.shape, matrix_news_2.shape

((9978, 9978), (9557, 9557), (26664, 9978), (21875, 9557))

Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [19]:
def generate_1(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen #для триграмм word2+' '+chosen
    
    return ' '.join(text)

In [20]:
def generate_2(matrix, id2word, bi2id, id2bi, n=100, start='<start> <start>'):
    text = []
    current_idx = bi2id[start]
    
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
          chosen = bi2id[start]
        else:
          word1, word2 = id2bi[current_idx].split()
          chosen = bi2id[word2 + ' ' + id2word[chosen]]
        current_idx = chosen
    
    return ' '.join(text)

In [21]:
print(generate_1(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

типа рарити а какая в webm 
 <start> <start> <start> в постсовковых бадишопах изо рта ещё крутой водочно-солевой раствор внутрь желудочного пуза и отнял их можно найти в поезде можно в больницах на 56 учеников значит 20 до гаммы 
 <start> иначе не специалист то и сая просается в uk есть ощущение что он привносит 
 ещё надеюсь сейчас жизнь а лучше алгебру решайте 
 существует аниме так уж плохо печатать могу туда и работал 
 берем эти delphi визуал васик кобол перл джява 
 это всратые как у них iq 80 слышь брат 
 а если после мака 



In [22]:
print(generate_2(matrix_dvach_2, id2word_dvach, bi2id_dvach, id2bi_dvach).replace('<end>', '\n'))

не плохая работа но твикстор слегка криповат 
 очевидная одинаковость этих двух структур происходит из одного правила правила построения перспективы 
 живот болит температура лейкоцитоз эмоциональный стресс ну ты ну ты ну привет 
 пизда короче 
 ну то же самое у бэйли джей спроси пополнение в клубе особо одаренных также верно получается если сделать 29 и 47 бесконечное число школьников в каждом новом клипе пробую разные эффекты и переходы 
 это кстати лучше чем пытаться побороть молчаливость менее времязатратно 
 никто не увидит 
 графика базы данных логические и функциональные языки есть отдельными предметами так-то я на втором курсе


In [23]:
print(generate_1(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

зато в 13 10 лет назад он не моглоне вызвать крупномасштабные инеожиданные жертвы в фбр 
 <start> не допустить их число пострадавших в настоящее время встречи с кпрф партия не подчинились приказу генпрокурора сша китая австралии прошли анти-индонезийские демонстрации данного сюжета представители фбр глава cbs мел кармазин уже в приморье и танков 
 <start> <start> по провинции несмотря на организм человека погибли два человека погибли двое террористов сам имеет смысла поскольку на соответствие проводимых ими была сочтена руководством министерства обороны россии во время может составить около половины своего участия в голову фбр называет также встреча волошина и сами участники акции


In [24]:
print(generate_2(matrix_news_2, id2word_news, bi2id_news, id2bi_news).replace('<end>', '\n'))

по словам эдуарда росселя на встрече с губернатором свердловской области 
 в корпус по сведениям агенства рейтер войдут подразделение сил безопасности в решении данных проблем отведена особая роль сегодня областной центр сахалина и курил получил статус очага распространения холеры 
 когда стало ясно что судно разламывается на части его капитан принял решение возобновить акцию гражданского неповиновения властям с требованием к властямускорить сооружение мемориала принцессе 
 отвечая на прямой вопрос собирается ливашингтон добиваться отсрочки решения по этой проблеме онасказала нет десять американских туристов и два жителя танзании погибли во время встречи 
 наши связи неразрывны мы будем идти по пути создания


Кажется, что тексты, сгенерированные триграммной моделью, лучше (как для новостей, так и для двача)

**Задание 2. (5 баллов)**

Напишите функцию оценивания нграммов на основе PMI. Используйте эту функцию вместо дефолтной в gensim.models. Phrases Обучите два последовательных модели Phrases с такой функцией и проанализируйте результаты, получаемые после преобразования текстов двумя Phrases.

Пояснения:
Формулу PMI можно посмотреть вот тут https://en.wikipedia.org/wiki/Pointwise_mutual_information , также там описано как вывести вероятности из количества вхождений слова1, слова2, нграмма и размера корпуса.
Чтобы функцию можно было поставить в аргумент scoring в Phrases у нее должны быть вот такие аргументы - scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count) 
Подберите параметр threshold под эту функцию. Чтобы проверить, что она вообще работает поставьте какое-то совсем маленькое число, чтобы порог проходили все нграммы и потом постепенно его повышайте.
В тетрадке есть пример обучения нескольких Phrases последовательно, воспользуйтесь им.


In [93]:
import math

def scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
  if bigram_count >= min_count:
    a_prob = worda_count / corpus_word_count
    b_prob = wordb_count / corpus_word_count
    bi_prob = bigram_count / corpus_word_count
    pmi_score = math.log(bi_prob/(a_prob*b_prob))    
  else:
    pmi_score = -1000000 
  
  return pmi_score

In [87]:
def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

In [88]:
import re
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import gensim

stops = set(stopwords.words('russian') + ["это", "весь"])
morph = MorphAnalyzer()

dvach_prepro = preprocess(dvach)
news_prepro = preprocess(news)

Посмотрим сначала на результаты для новостных текстов

In [134]:
#default (best results)
ph_n = gensim.models.Phrases(news_prepro)
p_n = gensim.models.phrases.Phraser(ph_n)

ph_n2 = gensim.models.Phrases(p_n[news_prepro])
p_n2 = gensim.models.phrases.Phraser(ph_n2)

for i in range(104,110):
  print(p_n2[p_n[news_prepro[i]]])

['как_сообщает', 'итар-тасс', 'со_ссылкой_на', 'пресс-центр', 'администрации', 'сахалинской_области', 'в', 'лечебных', 'учреждениях', 'южно-сахалинска', 'уже', 'находятсятся', '5', 'горожан', 'причем', 'у', 'двоих', 'из_них', 'болезнь', 'проходит', 'в', 'средне-тяжелой', 'форме']
['специалисты', 'госсанэпиднадзора', 'по', 'сахалинской_области', 'и', 'хабаровской', 'противочумной', 'станции', 'считают', 'что', 'возможным', 'источником', 'заражения', 'стала', 'река', 'хомутовка', 'в', 'военном', 'городке', 'и', 'слабосоленная', 'красная', 'рыба', 'собственного', 'приготовления', 'которую', 'жители', 'промывали', 'в', 'этой', 'реке']
['решением', 'губернатора', 'сахалинской_области', 'игоря', 'фархутдинова', 'сегодня', 'в', 'южно-сахалинске', 'начато', 'развертывание', 'специальных', 'госпиталей', 'и', 'изоляторов', 'для', 'локализации', 'и', 'ликвидации', 'очага', 'холеры']
['кроме_того', 'на_заседании', 'комиссии', 'было', 'принято', 'решение', 'о', 'трехкратном', 'бактериологическом', 

In [149]:
#default + threshold=0.005 (best results)
ph_n = gensim.models.Phrases(news_prepro, threshold = 0.005)
p_n = gensim.models.phrases.Phraser(ph_n)

ph_n2 = gensim.models.Phrases(p_n[news_prepro], threshold = 0.005)
p_n2 = gensim.models.phrases.Phraser(ph_n2)

for i in range(104,110):
  print(p_n2[p_n[news_prepro[i]]])

['как_сообщает', 'итар-тасс', 'со_ссылкой_на', 'пресс-центр', 'администрации', 'сахалинской_области', 'в', 'лечебных', 'учреждениях', 'южно-сахалинска', 'уже', 'находятсятся', '5', 'горожан', 'причем', 'у', 'двоих', 'из_них', 'болезнь', 'проходит', 'в', 'средне-тяжелой', 'форме']
['специалисты', 'госсанэпиднадзора', 'по', 'сахалинской_области', 'и', 'хабаровской', 'противочумной', 'станции', 'считают', 'что', 'возможным', 'источником', 'заражения', 'стала', 'река', 'хомутовка', 'в', 'военном', 'городке', 'и', 'слабосоленная', 'красная', 'рыба', 'собственного', 'приготовления', 'которую', 'жители', 'промывали', 'в', 'этой', 'реке']
['решением', 'губернатора', 'сахалинской_области', 'игоря', 'фархутдинова', 'сегодня_в', 'южно-сахалинске', 'начато', 'развертывание', 'специальных', 'госпиталей', 'и', 'изоляторов', 'для', 'локализации', 'и', 'ликвидации', 'очага', 'холеры']
['кроме_того', 'на_заседании', 'комиссии', 'было', 'принято', 'решение', 'о', 'трехкратном', 'бактериологическом', 'об

In [148]:
#pmi + threshold=0.005
ph_n = gensim.models.Phrases(news_prepro, scoring = scorer, threshold = 0.005)
p_n = gensim.models.phrases.Phraser(ph_n)

ph_n2 = gensim.models.Phrases(p_n[news_prepro], scoring = scorer, threshold = 0.005)
p_n2 = gensim.models.phrases.Phraser(ph_n2)

for i in range(104,110):
  print(p_n2[p_n[news_prepro[i]]])

['как_сообщает', 'итар-тасс_со', 'ссылкой_на', 'пресс-центр', 'администрации', 'сахалинской_области', 'в', 'лечебных', 'учреждениях', 'южно-сахалинска', 'уже', 'находятсятся', '5', 'горожан', 'причем', 'у', 'двоих', 'из_них', 'болезнь', 'проходит', 'в', 'средне-тяжелой', 'форме']
['специалисты', 'госсанэпиднадзора', 'по', 'сахалинской_области', 'и', 'хабаровской', 'противочумной', 'станции', 'считают', 'что', 'возможным', 'источником', 'заражения', 'стала', 'река', 'хомутовка', 'в', 'военном', 'городке', 'и', 'слабосоленная', 'красная', 'рыба', 'собственного', 'приготовления', 'которую', 'жители', 'промывали', 'в', 'этой', 'реке']
['решением', 'губернатора', 'сахалинской_области', 'игоря', 'фархутдинова', 'сегодня_в', 'южно-сахалинске', 'начато', 'развертывание', 'специальных', 'госпиталей', 'и', 'изоляторов', 'для', 'локализации', 'и', 'ликвидации', 'очага', 'холеры']
['кроме_того', 'на_заседании', 'комиссии', 'было', 'принято', 'решение', 'о', 'трехкратном', 'бактериологическом', 'об

In [127]:
#pmi + threshold=0.5
ph_n = gensim.models.Phrases(news_prepro, scoring = scorer, threshold = 0.5)
p_n = gensim.models.phrases.Phraser(ph_n)

ph_n2 = gensim.models.Phrases(p_n[news_prepro], scoring = scorer, threshold = 0.5)
p_n2 = gensim.models.phrases.Phraser(ph_n2)

for i in range(104,110):
  print(p_n2[p_n[news_prepro[i]]])

['как_сообщает', 'итар-тасс_со', 'ссылкой_на', 'пресс-центр', 'администрации', 'сахалинской_области', 'в', 'лечебных', 'учреждениях', 'южно-сахалинска', 'уже', 'находятсятся', '5', 'горожан', 'причем', 'у', 'двоих', 'из_них', 'болезнь', 'проходит', 'в', 'средне-тяжелой', 'форме']
['специалисты', 'госсанэпиднадзора', 'по', 'сахалинской_области', 'и', 'хабаровской', 'противочумной', 'станции', 'считают', 'что', 'возможным', 'источником', 'заражения', 'стала', 'река', 'хомутовка', 'в', 'военном', 'городке', 'и', 'слабосоленная', 'красная', 'рыба', 'собственного', 'приготовления', 'которую', 'жители', 'промывали', 'в', 'этой', 'реке']
['решением', 'губернатора', 'сахалинской_области', 'игоря', 'фархутдинова', 'сегодня_в', 'южно-сахалинске', 'начато', 'развертывание', 'специальных', 'госпиталей', 'и', 'изоляторов', 'для', 'локализации', 'и', 'ликвидации', 'очага', 'холеры']
['кроме_того', 'на_заседании', 'комиссии', 'было', 'принято', 'решение', 'о', 'трехкратном', 'бактериологическом', 'об

In [126]:
#pmi + threshold=6
ph_n = gensim.models.Phrases(news_prepro, scoring = scorer, threshold = 6)
p_n = gensim.models.phrases.Phraser(ph_n)

ph_n2 = gensim.models.Phrases(p_n[news_prepro], scoring = scorer, threshold = 6)
p_n2 = gensim.models.phrases.Phraser(ph_n2)

for i in range(104,110):
  print(p_n2[p_n[news_prepro[i]]])

['как', 'сообщает', 'итар-тасс', 'со_ссылкой', 'на', 'пресс-центр', 'администрации', 'сахалинской_области', 'в', 'лечебных', 'учреждениях', 'южно-сахалинска', 'уже', 'находятсятся', '5', 'горожан', 'причем', 'у', 'двоих', 'из', 'них', 'болезнь', 'проходит', 'в', 'средне-тяжелой', 'форме']
['специалисты', 'госсанэпиднадзора', 'по', 'сахалинской_области', 'и', 'хабаровской', 'противочумной', 'станции', 'считают', 'что', 'возможным', 'источником', 'заражения', 'стала', 'река', 'хомутовка', 'в', 'военном', 'городке', 'и', 'слабосоленная', 'красная', 'рыба', 'собственного', 'приготовления', 'которую', 'жители', 'промывали', 'в', 'этой', 'реке']
['решением', 'губернатора', 'сахалинской_области', 'игоря', 'фархутдинова', 'сегодня', 'в', 'южно-сахалинске', 'начато', 'развертывание', 'специальных', 'госпиталей', 'и', 'изоляторов', 'для', 'локализации', 'и', 'ликвидации', 'очага', 'холеры']
['кроме_того', 'на', 'заседании', 'комиссии', 'было', 'принято', 'решение', 'о', 'трехкратном', 'бактериол

In [143]:
#pmi + threshold=10
ph_n = gensim.models.Phrases(news_prepro, scoring = scorer, threshold = 10)
p_n = gensim.models.phrases.Phraser(ph_n)

ph_n2 = gensim.models.Phrases(p_n[news_prepro], scoring = scorer, threshold = 10)
p_n2 = gensim.models.phrases.Phraser(ph_n2)

for i in range(104,110):
  print(p_n2[p_n[news_prepro[i]]])

['как', 'сообщает', 'итар-тасс', 'со', 'ссылкой', 'на', 'пресс-центр', 'администрации', 'сахалинской', 'области', 'в', 'лечебных', 'учреждениях', 'южно-сахалинска', 'уже', 'находятсятся', '5', 'горожан', 'причем', 'у', 'двоих', 'из', 'них', 'болезнь', 'проходит', 'в', 'средне-тяжелой', 'форме']
['специалисты', 'госсанэпиднадзора', 'по', 'сахалинской', 'области', 'и', 'хабаровской', 'противочумной', 'станции', 'считают', 'что', 'возможным', 'источником', 'заражения', 'стала', 'река', 'хомутовка', 'в', 'военном', 'городке', 'и', 'слабосоленная', 'красная', 'рыба', 'собственного', 'приготовления', 'которую', 'жители', 'промывали', 'в', 'этой', 'реке']
['решением', 'губернатора', 'сахалинской', 'области', 'игоря', 'фархутдинова', 'сегодня', 'в', 'южно-сахалинске', 'начато', 'развертывание', 'специальных', 'госпиталей', 'и', 'изоляторов', 'для', 'локализации', 'и', 'ликвидации', 'очага', 'холеры']
['кроме', 'того', 'на', 'заседании', 'комиссии', 'было', 'принято', 'решение', 'о', 'трехкратн

Посмотрим на тексты двача

In [133]:
#default
ph_d = gensim.models.Phrases(dvach_prepro)
p_d = gensim.models.phrases.Phraser(ph_d)

ph_d2 = gensim.models.Phrases(p_d[dvach_prepro])
p_d2 = gensim.models.phrases.Phraser(ph_d2)

for i in range(110,120):
  print(p_d2[p_d[dvach_prepro[i]]])

['или', 'ты', 'про', 'кирхе']
['ты', 'превращаешься', 'в', 'суисейсеки', 'десу']
['я', 'думаю_что', 'он', 'про', 'кирхе']
['но', 'у', 'нее', 'и', 'так', 'есть', 'с_кем', 'мутить', 'а', 'гг', 'она', 'просто', 'траллирует', 'регулярно', 'потому_что', 'она', 'тру-рут']
['остальное', 'другие', 'руты', 'хуй_знает', 'зачем', 'засунутые', 'подряд']
['ну', 'хорошо', 'ты', 'пример', 'примеры', 'лучше', 'я', 'не', 'спорю']
['и', 'мне', 'и', 'гатари', 'нравятся', 'как', 'и', 'тамошняя', 'манера', 'подачи', 'просто', 'я', 'считаю', 'что', 'автор', 'мысль', 'доносил', 'на', 'примере', 'слабоватых', 'примеров', 'вот', 'и', 'все']
['эльфийка', 'тоже', 'ведь', 'со', 'своей', 'атмосферой', 'чего-то', 'не', 'в', 'меру', 'возвышенного']
['он', 'её', 'только', 'испортит']
['з', 'я', 'абсолютно', 'точно', 'про', 'табиту', 'но', 'нынче', 'вообще', 'грань', 'между', 'дандере', 'и', 'кудере', 'очень', 'размыта', 'так', 'что', 'я', 'не', 'пытаюсь', 'их', 'разделять', 'кардинально', 'чаще', 'всего']


In [140]:
#default + threshold (best results)
ph_d = gensim.models.Phrases(dvach_prepro, threshold = 0.005)
p_d = gensim.models.phrases.Phraser(ph_d)

ph_d2 = gensim.models.Phrases(p_d[dvach_prepro], threshold = 0.005)
p_d2 = gensim.models.phrases.Phraser(ph_d2)

for i in range(110,120):
  print(p_d2[p_d[dvach_prepro[i]]])

['или', 'ты', 'про', 'кирхе']
['ты', 'превращаешься', 'в', 'суисейсеки', 'десу']
['я_думаю', 'что_он', 'про', 'кирхе']
['но', 'у', 'нее', 'и_так', 'есть', 'с_кем', 'мутить', 'а', 'гг', 'она', 'просто', 'траллирует', 'регулярно', 'потому_что', 'она', 'тру-рут']
['остальное', 'другие', 'руты', 'хуй_знает', 'зачем', 'засунутые', 'подряд']
['ну', 'хорошо', 'ты', 'пример', 'примеры', 'лучше', 'я_не', 'спорю']
['и', 'мне', 'и', 'гатари', 'нравятся', 'как_и', 'тамошняя', 'манера', 'подачи', 'просто', 'я', 'считаю', 'что', 'автор', 'мысль', 'доносил', 'на', 'примере', 'слабоватых', 'примеров', 'вот_и', 'все']
['эльфийка', 'тоже', 'ведь', 'со', 'своей', 'атмосферой', 'чего-то', 'не_в', 'меру', 'возвышенного']
['он', 'её', 'только', 'испортит']
['з', 'я', 'абсолютно', 'точно', 'про', 'табиту', 'но', 'нынче', 'вообще', 'грань', 'между', 'дандере', 'и', 'кудере', 'очень', 'размыта', 'так_что', 'я_не', 'пытаюсь', 'их', 'разделять', 'кардинально', 'чаще', 'всего']


In [131]:
#pmi + threshold=0.005
ph_d = gensim.models.Phrases(dvach_prepro, scoring = scorer, threshold = 0.005)
p_d = gensim.models.phrases.Phraser(ph_d)

ph_d2 = gensim.models.Phrases(p_d[dvach_prepro], scoring = scorer, threshold = 0.005)
p_d2 = gensim.models.phrases.Phraser(ph_d2)

for i in range(110,120):
  print(p_d2[p_d[dvach_prepro[i]]])

['или', 'ты', 'про', 'кирхе']
['ты', 'превращаешься', 'в', 'суисейсеки', 'десу']
['я_думаю', 'что_он', 'про', 'кирхе']
['но', 'у', 'нее', 'и_так', 'есть', 'с_кем', 'мутить', 'а', 'гг', 'она', 'просто', 'траллирует', 'регулярно', 'потому_что', 'она', 'тру-рут']
['остальное', 'другие', 'руты', 'хуй_знает', 'зачем', 'засунутые', 'подряд']
['ну', 'хорошо', 'ты', 'пример', 'примеры', 'лучше', 'я_не', 'спорю']
['и', 'мне', 'и', 'гатари', 'нравятся', 'как', 'и', 'тамошняя', 'манера', 'подачи', 'просто', 'я', 'считаю', 'что', 'автор', 'мысль', 'доносил', 'на', 'примере', 'слабоватых', 'примеров', 'вот_и', 'все']
['эльфийка', 'тоже', 'ведь', 'со', 'своей', 'атмосферой', 'чего-то', 'не', 'в', 'меру', 'возвышенного']
['он', 'её', 'только', 'испортит']
['з', 'я', 'абсолютно', 'точно', 'про', 'табиту', 'но', 'нынче', 'вообще', 'грань', 'между', 'дандере', 'и', 'кудере', 'очень', 'размыта', 'так_что', 'я_не', 'пытаюсь', 'их', 'разделять', 'кардинально', 'чаще', 'всего']


In [147]:
#pmi + threshold=2
ph_d = gensim.models.Phrases(dvach_prepro, scoring = scorer, threshold = 2)
p_d = gensim.models.phrases.Phraser(ph_d)

ph_d2 = gensim.models.Phrases(p_d[dvach_prepro], scoring = scorer, threshold = 2)
p_d2 = gensim.models.phrases.Phraser(ph_d2)

for i in range(110,120):
  print(p_d2[p_d[dvach_prepro[i]]])

['или', 'ты', 'про', 'кирхе']
['ты', 'превращаешься', 'в', 'суисейсеки', 'десу']
['я_думаю', 'что_он', 'про', 'кирхе']
['но', 'у', 'нее', 'и', 'так', 'есть', 'с_кем', 'мутить', 'а', 'гг', 'она', 'просто', 'траллирует', 'регулярно', 'потому_что', 'она', 'тру-рут']
['остальное', 'другие', 'руты', 'хуй_знает', 'зачем', 'засунутые', 'подряд']
['ну', 'хорошо', 'ты', 'пример', 'примеры', 'лучше', 'я', 'не', 'спорю']
['и', 'мне', 'и', 'гатари', 'нравятся', 'как', 'и', 'тамошняя', 'манера', 'подачи', 'просто', 'я', 'считаю', 'что', 'автор', 'мысль', 'доносил', 'на', 'примере', 'слабоватых', 'примеров', 'вот', 'и', 'все']
['эльфийка', 'тоже', 'ведь', 'со', 'своей', 'атмосферой', 'чего-то', 'не', 'в', 'меру', 'возвышенного']
['он', 'её', 'только', 'испортит']
['з', 'я', 'абсолютно', 'точно', 'про', 'табиту', 'но', 'нынче', 'вообще', 'грань', 'между', 'дандере', 'и', 'кудере', 'очень', 'размыта', 'так_что', 'я', 'не', 'пытаюсь', 'их', 'разделять', 'кардинально', 'чаще', 'всего']


In [146]:
#pmi + threshold = 6
ph_d = gensim.models.Phrases(dvach_prepro, scoring = scorer, threshold = 6)
p_d = gensim.models.phrases.Phraser(ph_d)

ph_d2 = gensim.models.Phrases(p_d[dvach_prepro], scoring = scorer, threshold = 6)
p_d2 = gensim.models.phrases.Phraser(ph_d2)

for i in range(110,120):
  print(p_d2[p_d[dvach_prepro[i]]])

['или', 'ты', 'про', 'кирхе']
['ты', 'превращаешься', 'в', 'суисейсеки', 'десу']
['я', 'думаю', 'что', 'он', 'про', 'кирхе']
['но', 'у', 'нее', 'и', 'так', 'есть', 'с', 'кем', 'мутить', 'а', 'гг', 'она', 'просто', 'траллирует', 'регулярно', 'потому', 'что', 'она', 'тру-рут']
['остальное', 'другие', 'руты', 'хуй', 'знает', 'зачем', 'засунутые', 'подряд']
['ну', 'хорошо', 'ты', 'пример', 'примеры', 'лучше', 'я', 'не', 'спорю']
['и', 'мне', 'и', 'гатари', 'нравятся', 'как', 'и', 'тамошняя', 'манера', 'подачи', 'просто', 'я', 'считаю', 'что', 'автор', 'мысль', 'доносил', 'на', 'примере', 'слабоватых', 'примеров', 'вот', 'и', 'все']
['эльфийка', 'тоже', 'ведь', 'со', 'своей', 'атмосферой', 'чего-то', 'не', 'в', 'меру', 'возвышенного']
['он', 'её', 'только', 'испортит']
['з', 'я', 'абсолютно', 'точно', 'про', 'табиту', 'но', 'нынче', 'вообще', 'грань', 'между', 'дандере', 'и', 'кудере', 'очень', 'размыта', 'так', 'что', 'я', 'не', 'пытаюсь', 'их', 'разделять', 'кардинально', 'чаще', 'всего']

Можно отметить, что для новостных текстов при pmi threshold <=6 (примерно), все еще дает неплохие результаты, в то время как для текстов двача, уже при threshold >= 2 (примерно), количество выделенных биграмм начинает уменьшаться

Однако в целом дефолтный scorer дает результаты лучше, чем pmi